In [1]:
!pip install langchain langchain-community langchain-core transformers==4.52.4

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 91.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 92.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 1.2.19
    Uninstalling SQLAlchemy-1.2.19:
      Successfully uninstalled SQLAlchemy-1.2.19
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
import torch
import re

model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

def generate_text(prompt, max_length=1000, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

2026-01-05 19:40:15.566271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767642016.009126      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767642016.124251      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767642017.190366      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767642017.190406      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767642017.190408      55 computation_placer.cc:177] computation placer alr

...

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [46]:
def get_pdf_text(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        return "\n".join(page.extract_text() for page in pdf.pages)

# تعريف الـ Schema
response_schemas = [
    ResponseSchema(name="full_name", description="The full name of the candidate as a string"),
    ResponseSchema(name="email", description="The email address of the candidate as a string"),
    ResponseSchema(name="education", description="A list of education records, each containing degree, institution, and year"),
    ResponseSchema(name="skills", description="A list of skills as strings"),
    ResponseSchema(name="experience", description="A list of work experience records, each containing role, company, and years")
]

In [47]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [48]:
pdf_path = "/kaggle/input/cv-output-parser/My CV.pdf"
pdf_text = get_pdf_text(pdf_path)

In [50]:
# إنشاء الـ Prompt
prompt = f"""Extract information from this resume and return only valid JSON:

Resume Text:
{pdf_text}

Return JSON in this format:
{{
  "full_name": "full name here",
  "email": "email here",
  "education": [{{"degree": "", "institution": "", "year": ""}}],
  "skills": ["skill1", "skill2"],
  "experience": [{{"role": "", "company": "", "years": ""}}]
}}

JSON Output:"""

# توليد النتيجة مع max_length أكبر
print("جاري تحليل السيرة الذاتية...")
result = generate_text(prompt, max_length=3000)

if isinstance(result, list):
    result = result[0]

print("تم الانتهاء!\n")

# استخراج JSON
def extract_json(text):
    # نبحث عن آخر JSON في النص (اللي بعد "JSON Output:")
    parts = text.split("JSON Output:")
    if len(parts) > 1:
        json_part = parts[-1]
    else:
        json_part = text
    
    start = json_part.find('{')
    end = json_part.rfind('}')
    
    if start != -1 and end != -1:
        return json_part[start:end+1]
    
    return None

json_text = extract_json(result)

if json_text:
    try:
        output_data = json.loads(json_text)
        
        print("===== البيانات المستخرجة =====\n")
        print("الاسم:", output_data.get("full_name"))
        print("البريد:", output_data.get("email"))
        print("\nالتعليم:")
        for edu in output_data.get("education", []):
            print(f"  - {edu}")
        print("\nالمهارات:")
        for skill in output_data.get("skills", [])[:15]:
            print(f"  - {skill}")
        print("\nالخبرات:")
        for exp in output_data.get("experience", []):
            print(f"  - {exp}")
        
        print("\n===== JSON =====")
        print(json.dumps(output_data, indent=2, ensure_ascii=False))
        
    except Exception as e:
        print(f"خطأ: {e}")
        print("\nالنتيجة الكاملة:")
        print(result)
else:
    print("مفيش JSON!")
    print(result[:1000])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


جاري تحليل السيرة الذاتية...
تم الانتهاء!

===== البيانات المستخرجة =====

الاسم: Ahmed Hussien Mostafa
البريد: hosdinmhommed@gmail.com

التعليم:
  - {'degree': 'Bachelor of Computer Science', 'institution': 'Ahram Canadian University', 'year': '2021 – 2025'}

المهارات:
  - Python
  - SQL
  - C++
  - TensorFlow
  - PyTorch
  - Git
  - GitHub
  - Hugging Face
  - Roboflow
  - Machine Learning
  - Deep Learning
  - Computer Vision
  - NLP
  - Jupyter Notebook
  - Pandas

الخبرات:

===== JSON =====
{
  "full_name": "Ahmed Hussien Mostafa",
  "email": "hosdinmhommed@gmail.com",
  "education": [
    {
      "degree": "Bachelor of Computer Science",
      "institution": "Ahram Canadian University",
      "year": "2021 – 2025"
    }
  ],
  "skills": [
    "Python",
    "SQL",
    "C++",
    "TensorFlow",
    "PyTorch",
    "Git",
    "GitHub",
    "Hugging Face",
    "Roboflow",
    "Machine Learning",
    "Deep Learning",
    "Computer Vision",
    "NLP",
    "Jupyter Notebook",
    "Pandas"

In [56]:
def extract_json_block(text):
    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)
    
    if matches:
        return f"```json\n{matches[-1]}\n```"
    return None

# الاستخدام
formatted_json = extract_json_block(result)
if formatted_json:
    print(formatted_json)
else:
    # لو مافيش ```json في النتيجة، نعمله بنفسنا
    if json_text:
        output_data = json.loads(json_text)
        json_output = json.dumps(output_data, indent=2, ensure_ascii=False)
        print(f"```json\n{json_output}\n```")

```json
{
  "full_name": "Ahmed Hussien Mostafa",
  "email": "hosdinmhommed@gmail.com",
  "education": [
    {
      "degree": "Bachelor of Computer Science",
      "institution": "Ahram Canadian University",
      "year": "2021 – 2025"
    }
  ],
  "skills": [
    "Python",
    "SQL",
    "C++",
    "TensorFlow",
    "PyTorch",
    "Git",
    "GitHub",
    "Hugging Face",
    "Roboflow",
    "Machine Learning",
    "Deep Learning",
    "Computer Vision",
    "NLP",
    "Jupyter Notebook",
    "Pandas",
    "NumPy",
    "Matplotlib",
    "Seaborn",
    "Problem-Solving",
    "Adaptability",
    "Teamwork"
  ],
  "experience": []
}
```


In [60]:
final_json=(f"```json\n{json_output}\n```") 
print(final_json)

```json
{
  "full_name": "Ahmed Hussien Mostafa",
  "email": "hosdinmhommed@gmail.com",
  "education": [
    {
      "degree": "Bachelor of Computer Science",
      "institution": "Ahram Canadian University",
      "year": "2021 – 2025"
    }
  ],
  "skills": [
    "Python",
    "SQL",
    "C++",
    "TensorFlow",
    "PyTorch",
    "Git",
    "GitHub",
    "Hugging Face",
    "Roboflow",
    "Machine Learning",
    "Deep Learning",
    "Computer Vision",
    "NLP",
    "Jupyter Notebook",
    "Pandas",
    "NumPy",
    "Matplotlib",
    "Seaborn",
    "Problem-Solving",
    "Adaptability",
    "Teamwork"
  ],
  "experience": []
}
```


In [61]:
result=output_parser.parse(final_json)
print(result)

{'full_name': 'Ahmed Hussien Mostafa', 'email': 'hosdinmhommed@gmail.com', 'education': [{'degree': 'Bachelor of Computer Science', 'institution': 'Ahram Canadian University', 'year': '2021 – 2025'}], 'skills': ['Python', 'SQL', 'C++', 'TensorFlow', 'PyTorch', 'Git', 'GitHub', 'Hugging Face', 'Roboflow', 'Machine Learning', 'Deep Learning', 'Computer Vision', 'NLP', 'Jupyter Notebook', 'Pandas', 'NumPy', 'Matplotlib', 'Seaborn', 'Problem-Solving', 'Adaptability', 'Teamwork'], 'experience': []}


In [62]:
print(type(result))

<class 'dict'>


In [63]:
result["email"]

'hosdinmhommed@gmail.com'